In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_5%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,NaN,0.0,0.2,0.0,2.4,8.574,0.4,0.5,NaN,1.0
1,NaN,0.00,NaN,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,NaN,10.80,0.6,6.480,NaN,6.0,0.6,...,0.4,0.0,0.2,NaN,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,NaN,0.2,...,0.4,NaN,NaN,0.0,2.0,1.416,0.4,0.5,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,NaN,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0.0
773,55.57,0.24,0.2,NaN,0.04,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.2,0.040,0.00,0.2,0.000,0.20,NaN,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      56
PARA_A            46
Score_A           38
Risk_A            46
PARA_B            43
Score_B           39
Risk_B            46
TOTAL             34
numbers           38
Score_B.1         26
Risk_C            41
Money_Value       43
Score_MV          35
Risk_D            35
District_Loss     24
PROB              45
RiSk_E            48
History           48
Prob              43
Risk_F            40
Score             37
Inherent_Risk     33
CONTROL_RISK      35
Detection_Risk    42
Audit_Risk        49
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 1010
persentase : 5.005947660586836


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score      7.216495
PARA_A            5.927835
Score_A           4.896907
Risk_A            5.927835
PARA_B            5.541237
Score_B           5.025773
Risk_B            5.927835
TOTAL             4.381443
numbers           4.896907
Score_B.1         3.350515
Risk_C            5.283505
Money_Value       5.541237
Score_MV          4.510309
Risk_D            4.510309
District_Loss     3.092784
PROB              5.798969
RiSk_E            6.185567
History           6.185567
Prob              5.541237
Risk_F            5.154639
Score             4.768041
Inherent_Risk     4.252577
CONTROL_RISK      4.510309
Detection_Risk    5.412371
Audit_Risk        6.314433
Risk              0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,7)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,...,0.0,0.0,0.2,0.0,2.4,8.574,0.4,0.5,0.0000,1.0
1,0.00,0.00,0.0,0.000,4.83,0.2,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,0.00,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,0.0,0.2,...,0.4,0.0,0.0,0.0,2.0,1.416,0.4,0.5,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.49,0.2,0.098,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.0,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.2,0.074,0.84,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.3136,0.0
773,55.57,0.24,0.2,0.000,0.04,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.2,0.040,0.00,0.2,0.000,0.20,0.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.440,0.4,0.5,0.2880,0.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4372


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[3.89, 55.57, 55.57, 55.57, 3.89, 2.37, 55.57]
mean :  33.20428571428571
standar deviasi :  25.829962035889835
fuzzy :  [(3.89, 0.5252262478562792), (55.57, 0.6874010024062931), (55.57, 0.6874010024062931), (55.57, 0.6874010024062931), (3.89, 0.5252262478562792), (2.37, 0.4904485208092542), (55.57, 0.6874010024062931)]
fuzzy :  None
pembilang :  158.0441180175106
penyebut :  4.2905050261469855
rata-rata tertimbang :  36.83578437838108
 
[2.72, 3.89, 3.41, 3.41, 3.41, 2.72, 2.72]
mean :  3.1828571428571424
standar deviasi :  0.4305383435956923
fuzzy :  [(2.72, 0.5611182818306036), (3.89, 0.25957728127968377), (3.41, 0.8700929711817762), (3.41, 0.8700929711817762), (3.41, 0.8700929711817762), (2.72, 0.5611182818306036), (2.72, 0.5611182818306036)]
fuzzy :  None
pembilang :  14.489531899105264
penyebut :  4.553211040316824
rata-rata tertimbang :  3.182266706024908
 
[2.72, 2.72, 3.89, 3.41, 3.89, 3.89, 0.0]
mean :  2.9314285714285715
standar deviasi :  1.2910840248348183
fuzzy :

[55.57, 1.99, 55.57, 2.72, 55.57, 55.57, 0.0]
mean :  32.427142857142854
standar deviasi :  26.733662415061072
fuzzy :  [(55.57, 0.6875202249019042), (1.99, 0.5230590585899626), (55.57, 0.6875202249019042), (2.72, 0.5393731304952981), (55.57, 0.6875202249019042), (55.57, 0.6875202249019042), (0.0, 0.479231070531371)]
fuzzy :  None
pembilang :  155.3299780327365
penyebut :  4.291744159224249
rata-rata tertimbang :  36.192739424806035
 
[0.0, 55.57, 2.37, 55.57, 55.57, 2.37, 55.57]
mean :  32.43142857142857
standar deviasi :  26.728129873423846
fuzzy :  [(0.0, 0.47899195802710876), (55.57, 0.687509009595634), (2.37, 0.5313033566158689), (55.57, 0.687509009595634), (55.57, 0.687509009595634), (2.37, 0.5313033566158689), (55.57, 0.687509009595634)]
fuzzy :  None
pembilang :  155.33788056327674
penyebut :  4.291634709641382
rata-rata tertimbang :  36.19550382848336
 
[0.0, 55.57, 3.89, 2.37, 55.57, 1.85, 2.37]
mean :  17.374285714285715
standar deviasi :  24.180209060388243
fuzzy :  [(0.0, 

[0.0, 21.61, 21.61, 2.37, 2.37, 55.57, 55.57]
mean :  22.728571428571428
standar deviasi :  22.377131457023218
fuzzy :  [(0.0, 0.5970376134526953), (21.61, 0.99875154655856), (21.61, 0.99875154655856), (2.37, 0.6611204020913873), (2.37, 0.6611204020913873), (55.57, 0.34066270262301884), (55.57, 0.34066270262301884)]
fuzzy :  None
pembilang :  84.16100531769645
penyebut :  4.598106915998627
rata-rata tertimbang :  18.303403303839485
 
[0.0, 3.89, 2.927258271104354, 2.72, 3.41, 3.89, 3.89]
mean :  2.9610368958720508
standar deviasi :  1.288269298307935
fuzzy :  [(0.0, 0.07127655090754753), (3.89, 0.7710814771968824), (2.927258271104354, 0.9996563463565017), (2.72, 0.9826506255508197), (3.41, 0.9410865492892889), (3.89, 0.7710814771968824), (3.89, 0.7710814771968824)]
fuzzy :  None
pembilang :  17.80668798159635
penyebut :  5.307914503694805
rata-rata tertimbang :  3.3547428032612863
 
[0.0, 59.85, 1.85, 55.57, 55.57, 3.41, 0.0]
mean :  25.178571428571427
standar deviasi :  27.60796905065

[3.14, 1.94, 0.0, 1.44, 4.18, 0.45, 1.9]
mean :  1.864285714285714
standar deviasi :  1.3444473866649467
fuzzy :  [(3.14, 0.6375411491689991), (1.94, 0.9984156586889806), (0.0, 0.382392612344339), (1.44, 0.9514278901051647), (4.18, 0.22690453585632894), (0.45, 0.5750849198914186), (1.9, 0.9996472681458978)]
fuzzy :  None
pembilang :  8.415440731306516
penyebut :  4.771414034201128
rata-rata tertimbang :  1.7637204968978348
 
[0.07, 0.77, 1.05, 0.0, 1.29, 1.25, 1.5]
mean :  0.8471428571428571
standar deviasi :  0.5546353830790688
fuzzy :  [(0.07, 0.3747305622090451), (0.77, 0.9903749546319656), (1.05, 0.9353082708158583), (0.0, 0.31150666803379434), (1.29, 0.727064258552335), (1.25, 0.7681564614280719), (1.5, 0.5002228736149452)]
fuzzy :  None
pembilang :  4.419336319517918
penyebut :  4.607364049286016
rata-rata tertimbang :  0.9591897389143288
 
[0.0, 0.0, 0.0, 0.0, 1.65, 0.53, 1.27]
mean :  0.4928571428571428
standar deviasi :  0.6454171250486946
fuzzy :  [(0.0, 0.7471181648185071), 

[0.47, 1.6, 0.006206571476135272, 0.005320754229251528, 0.0, 0.0, 0.0]
mean :  0.2973610465293411
standar deviasi :  0.5557493895638165
fuzzy :  [(0.47, 0.9529011169654366), (1.6, 0.06413759843531011), (0.006206571476135272, 0.8717793756285837), (0.005320754229251528, 0.8710506739790298), (0.0, 0.8666401227234808), (0.0, 0.8666401227234808), (0.0, 0.8666401227234808)]
fuzzy :  None
pembilang :  0.5605290900339771
penyebut :  5.359789133178802
rata-rata tertimbang :  0.10458043704818973
 
[6.67, 6.31, 1.65, 2.18, 3.48, 2.86, 0.0]
mean :  3.3071428571428574
standar deviasi :  2.2522859136296827
fuzzy :  [(6.67, 0.32806690513287395), (6.31, 0.4111966043331824), (1.65, 0.7628883090468733), (2.18, 0.8823124010013806), (3.48, 0.9970595533901792), (2.86, 0.9804881376186181), (0.0, 0.3403048899472881)]
fuzzy :  None
pembilang :  14.239026894076071
penyebut :  4.7023168004703955
rata-rata tertimbang :  3.028087536052797
 
[4.34, 1.7637204968978348, 1.97, 2.39, 1.19, 1.9, 0.0]
mean :  1.93624578

...

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimba

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimba

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimba

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimba

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  4
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimba

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.2,0.500,6.680000,5.000000,0.2,...,0.396424,0.0,0.2,0.0,2.4,8.574,0.4,0.500000,1.256343,1.000000
1,36.835784,0.999674,0.267571,0.303674,4.83,0.2,0.966,4.830000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,2.554,0.4,0.500000,0.510800,0.008941
2,3.890000,0.510000,0.200000,0.102000,0.23,0.2,0.046,0.740000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.548,0.4,0.500000,0.309600,0.000000
3,3.890000,1.763720,0.200000,0.843913,10.80,0.6,6.480,11.079904,6.000000,0.6,...,0.400000,0.0,0.2,0.0,4.4,17.530,0.4,0.500000,3.506000,1.000000
4,3.890000,0.959190,0.200000,0.338889,0.08,0.2,0.016,0.080000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.416,0.4,0.500000,0.283200,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,52.478836,0.490000,0.200000,0.098000,0.40,0.2,0.080,0.890000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.578,0.4,0.495529,0.315600,0.000000
772,55.570000,0.470000,0.200000,0.094000,0.37,0.2,0.074,0.840000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.568,0.4,0.500000,0.313600,0.000000
773,55.570000,0.240000,0.200000,0.000000,0.04,0.2,0.008,0.280000,5.000000,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.456,0.4,0.500000,0.291200,0.000000
774,55.570000,0.200000,0.200000,0.040000,0.00,0.2,0.000,0.200000,4.955295,0.2,...,0.400000,0.0,0.2,0.0,2.0,1.440,0.4,0.500000,0.288000,0.000000


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
count,776.00000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,19.85047,2.372224,0.351686,1.363166,10.789837,0.312839,6.339992,13.248115,5.063612,0.223395,...,0.513692,0.100515,0.216549,0.052577,2.689665,17.591830,0.569266,0.499633,6.891089,0.393053
std,23.78650,4.706899,0.173675,3.427914,50.029733,0.169472,30.076244,51.314690,0.261634,0.079286,...,0.280095,0.528125,0.068048,0.305339,0.846708,54.714992,0.441396,0.003315,38.277887,0.488732
min,1.85000,0.000000,0.128182,0.000000,0.000000,0.170301,0.000000,0.000000,4.386002,0.170301,...,0.256364,0.000000,0.128182,0.000000,1.982118,1.400000,0.350880,0.438600,0.182831,0.000000
25%,2.37000,0.328236,0.200000,0.055500,0.000000,0.200000,0.000000,0.587500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.584000,0.400000,0.500000,0.315600,0.000000
50%,3.89000,0.950267,0.200000,0.187000,0.385508,0.200000,0.071000,1.410000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.225222,2.200000,0.400000,0.500000,0.511200,0.000000
75%,55.57000,2.607500,0.600000,1.528500,4.212500,0.400000,1.785000,7.767500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,3.200000,10.663500,0.400000,0.500000,3.137497,1.000000
max,59.85000,85.000000,0.600000,51.000000,1264.630000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,2.400000,9.000000,0.600000,5.400000,5.200000,801.262000,5.800000,0.500000,961.514400,1.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[120   4]
 [  1  69]]
0.9742268041237113


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[123   1]
 [  1  69]]
0.9896907216494846


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
